In [12]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer

In [3]:
import os
from huggingface_hub import login

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")
login(token=huggingface_token)

## Embed into Database

In [15]:
# Step 1: Load the datasets
def load_data():
    """Load the podcast data and ground truth data."""
    # Load podcast data from JSON
    with open('podcast_topic.json', 'r', encoding='utf-8') as f:
        podcast_topic = json.load(f)
    
    # Load ground truth data from JSON
    with open('podcast_ground_truth.json', 'r', encoding='utf-8') as f:
        ground_truth = json.load(f)
    
    return podcast_topic, ground_truth

In [16]:
# Step 2: Initialize the embedding model
def init_model(model_name="intfloat/multilingual-e5-base"):
    """Initialize the text embedding model."""
    model = SentenceTransformer(model_name)
    
    return model

In [3]:
# Step 3: Create a Qdrant collection and upload podcast data
def setup_qdrant(podcast_topic, model, collection_name="podcast_collection"):
    """Create a Qdrant collection and upload podcast vectors."""
    # Initialize Qdrant client
    client = QdrantClient(url="http://localhost:6333")

    # Get vector dimension with a sample embedding
    # sample_text = "Sample text to determine vector dimensions"
    # sample_vector = model.encode(sample_text)
    # vector_size = len(sample_vector)
    
    # Try to create collection (ignore if exists)
    try:
        client.create_collection(
            collection_name=collection_name,
            vectors_config=models.VectorParams(
                size=model.get_sentence_embedding_dimension(),
                distance=models.Distance.COSINE
            )
        )
        print(f"Created new collection: {collection_name}")
    except Exception as e:
        print(f"Collection may already exist: {e}")
    
    # Embed and upload each podcast episode
    points = []
    for episode_id, content in tqdm(podcast_topic.items(), desc="Embedding podcast topics"):
        # Get topics and song if available
        topics = content.get('topic', [])
        song = content.get('song', "")
        
        # Create a text representation combining all topics
        topics_text = '; '.join(topics) if topics else ""
        
        # Add song if available and not 'null' or NaN
        full_text = topics_text
        if song and song != "null" and song != "nan":
            full_text += f"; Song: {song}"
        
        # Skip if no text to embed
        if not full_text:
            continue
        
        # Embed the text
        embedding = model.encode(full_text)
        
        # Create a point
        points.append(
            models.PointStruct(
                id=int(episode_id),
                vector=embedding.tolist(),
                payload={
                    'episode': int(episode_id),
                    'topics': topics,
                    'song': song
                }
            )
        )
    
    # Upload points in batches
    batch_size = 100
    for i in range(0, len(points), batch_size):
        batch = points[i:i+batch_size]
        client.upsert(
            collection_name=collection_name,
            points=batch
        )
    
    print(f"Uploaded {len(points)} episodes to Qdrant.")
    return client

In [20]:
# Step 4: Evaluate retrieval using ground truth data
def evaluate_retrieval(client, model, ground_truth, collection_name="podcast_collection", k=5):
    """
    Evaluate retrieval performance using ground truth data.
    
    For each sentence in the ground truth data, embed it and query Qdrant.
    Check if the correct episode is in the top k results.
    """
    # Prepare to store results
    all_relevance = []
    
    # Process each episode and its data
    for episode_id, episode_data in tqdm(ground_truth.items(), desc="Evaluating episodes"):
        words = episode_data.get('word', [])
        
        for word in words:
            # Embed the sentence
            query_vector = model.encode(word)
            
            # Query Qdrant
            search_results = client.search(
                collection_name=collection_name,
                query_vector=query_vector.tolist(),
                limit=k
            )
            
            # Check if the correct episode is in the results
            result_ids = [result.id for result in search_results]
            relevance = [int(episode_id) == result_id for result_id in result_ids]
            
            all_relevance.append(relevance)
    
    return all_relevance

In [5]:
# Step 5: Calculate metrics
def calculate_metrics(relevance_results):
    """Calculate hit rate and MRR from relevance results."""
    # Hit rate (Recall@k)
    hit_count = sum(1 for relevance in relevance_results if True in relevance)
    hit_rate = hit_count / len(relevance_results) if relevance_results else 0
    
    # Mean Reciprocal Rank (MRR)
    mrr_sum = 0
    for relevance in relevance_results:
        for i, is_relevant in enumerate(relevance):
            if is_relevant:
                mrr_sum += 1 / (i + 1)
                break
    
    mrr = mrr_sum / len(relevance_results) if relevance_results else 0
    
    return {
        'hit_rate': hit_rate,
        'mrr': mrr,
        'total_queries': len(relevance_results),
        'successful_queries': hit_count
    }

In [17]:
# Step 6: Evaluate for different k values
def evaluate_multiple_k(client, model, ground_truth, collection_name="podcast_collection", k_values=[3, 5, 10]):
    """Evaluate retrieval for different k values."""
    results = {}
    
    for k in k_values:
        print(f"\nEvaluating for k={k}")
        relevance_results = evaluate_retrieval(client, model, ground_truth, collection_name, k)
        metrics = calculate_metrics(relevance_results)
        
        results[k] = metrics
        print(f"Hit Rate: {metrics['hit_rate']:.4f}, MRR: {metrics['mrr']:.4f}")
    
    return results

In [19]:
print("Loading data...")
podcast_topic, ground_truth = load_data()

# Initialize model
model = init_model()

# Setup Qdrant
print("Setting up Qdrant...")
client = setup_qdrant(podcast_topic, model)

# Evaluate for different k values
print("Evaluating search performance...")
results = evaluate_multiple_k(client, model, ground_truth)

# Print final results
print("\nFinal Evaluation Results:")
print("========================")
for k, metrics in results.items():
    print(f"k={k}:")
    print(f"  Hit Rate: {metrics['hit_rate']:.4f}")
    print(f"  MRR:      {metrics['mrr']:.4f}")
    print(f"  ({metrics['successful_queries']} out of {metrics['total_queries']} queries successful)")

# Save results to file
with open('evaluation_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\nResults saved to evaluation_results.json")

Loading data...


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

...

Hit Rate: 0.0787, MRR: 0.0543

Evaluating for k=5


Evaluating episodes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 411/411 [00:04<00:00, 88.39it/s]


Hit Rate: 0.1573, MRR: 0.0723

Evaluating for k=10


Evaluating episodes: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 411/411 [00:04<00:00, 88.78it/s]

Hit Rate: 0.2472, MRR: 0.0847

Final Evaluation Results:
k=3:
  Hit Rate: 0.0787
  MRR:      0.0543
  (7 out of 89 queries successful)
k=5:
  Hit Rate: 0.1573
  MRR:      0.0723
  (14 out of 89 queries successful)
k=10:
  Hit Rate: 0.2472
  MRR:      0.0847
  (22 out of 89 queries successful)

Results saved to evaluation_results.json


In [21]:
# Evaluate for different k values
print("Evaluating search performance...")
results = evaluate_multiple_k(client, model, ground_truth)

# Print final results
print("\nFinal Evaluation Results:")
print("========================")
for k, metrics in results.items():
    print(f"k={k}:")
    print(f"  Hit Rate: {metrics['hit_rate']:.4f}")
    print(f"  MRR:      {metrics['mrr']:.4f}")
    print(f"  ({metrics['successful_queries']} out of {metrics['total_queries']} queries successful)")

# Save results to file
with open('evaluation_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\nResults saved to evaluation_results.json")

Evaluating search performance...

Evaluating for k=3


Evaluating episodes:   0%|                                                                      | 0/411 [00:00<?, ?it/s]/tmp/ipykernel_27447/3006615302.py:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(
Evaluating episodes: 100%|████████████████████████████████████████████████████████████| 411/411 [01:43<00:00,  3.97it/s]


Hit Rate: 0.5125, MRR: 0.4268

Evaluating for k=5


Evaluating episodes: 100%|████████████████████████████████████████████████████████████| 411/411 [01:43<00:00,  3.99it/s]


Hit Rate: 0.5687, MRR: 0.4396

Evaluating for k=10


Evaluating episodes: 100%|████████████████████████████████████████████████████████████| 411/411 [01:43<00:00,  3.98it/s]

Hit Rate: 0.6518, MRR: 0.4507

Final Evaluation Results:
k=3:
  Hit Rate: 0.5125
  MRR:      0.4268
  (1048 out of 2045 queries successful)
k=5:
  Hit Rate: 0.5687
  MRR:      0.4396
  (1163 out of 2045 queries successful)
k=10:
  Hit Rate: 0.6518
  MRR:      0.4507
  (1333 out of 2045 queries successful)

Results saved to evaluation_results.json


### Use the BAAI/bge-base-en-v1.5 from FastEmbed

In [6]:
client = QdrantClient(url="http://localhost:6333")

In [7]:
model = TextEmbedding(model_name="BAAI/bge-base-en-v1.5")
print("The model BAAI/bge-small-en-v1.5 is ready to use.")

False

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

The model BAAI/bge-small-en-v1.5 is ready to use.


In [8]:
with open('podcast_topic.json', 'r', encoding='utf-8') as f:
    podcast_topic = json.load(f)

In [22]:
def json_to_points(json_data):
    """
    Converts the podcast episode JSON data into separate text strings for each episode.
    
    Args:
        json_data (dict): JSON dictionary containing episode data
        
    Returns:
        points: 
    """
    points = []
    
    for episode_id, content in tqdm(json_data.items()):
        episode_text = []
        
        # Add topics if available
        if 'topic' in content and content['topic']:
            topics_text = ', '.join(content['topic'])
            episode_text.append(topics_text)
        
        # Add song if available
        if 'song' in content and content['song']:
            episode_text.append(f"Song: {content['song']}")

        embedding = list(model.embed('\n'.join(episode_text)))[0]
        
        points.append(
            models.PointStruct(
                id=int(episode_id),
                vector=embedding
            )
        )
        
    return points

In [18]:
# Try to create collection (ignore if exists)
try:
    client.create_collection(
        collection_name="podcast_topic_collection",
        vectors_config=models.VectorParams(
            size=768,
            distance=models.Distance.COSINE
        )
    )
    print(f"Created new collection: podcast_topic_collection")
except Exception as e:
    print(f"Collection may already exist: {e}")

Created new collection: podcast_topic_collection


In [23]:
embed_points = json_to_points(podcast_topic)

100%|███████████████████████████████████████████████████████████████████████| 411/411 [00:57<00:00,  7.11it/s]


In [24]:
client.upsert(
    collection_name="podcast_topic_collection",
    points=embed_points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [4]:
client = QdrantClient(url="http://localhost:6333")

# Get vector dimension
sample_text = f"title:{df.iloc[0]['title']}; summary:{df.iloc[0]['summary']}"
sample_vector = list(model.embed(sample_text))[0]
vector_size = len(sample_vector)

# Try to create collection (ignore if exists)
try:
    client.create_collection(
        collection_name="podcast_collection",
        vectors_config=models.VectorParams(
            size=vector_size,
            distance=models.Distance.COSINE
        )
    )
    print(f"Created new collection: podcast_collection")
except Exception as e:
    print(f"Collection may already exist: {e}")

NameError: name 'model' is not defined

In [13]:
client.upsert(
    collection_name="podcast_collection",
    points=embed_points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# def setup_qdrant(podcast_df, model, collection_name="podcast_collection"):
#     """Create a Qdrant collection and upload podcast vectors."""
#     # Initialize Qdrant client
#     client = QdrantClient(url="http://localhost:6333")
    
#     # Get vector dimension
#     sample_text = f"title:{podcast_df.iloc[0]['title']}; summary:{podcast_df.iloc[0]['summary']}"
#     sample_vector = list(model.embed(sample_text))[0]
#     vector_size = len(sample_vector)
    
#     # Try to create collection (ignore if exists)
#     try:
#         client.create_collection(
#             collection_name=collection_name,
#             vectors_config=models.VectorParams(
#                 size=vector_size,
#                 distance=models.Distance.COSINE
#             )
#         )
#         print(f"Created new collection: {collection_name}")
#     except Exception as e:
#         print(f"Collection may already exist: {e}")
    
#     # Embed and upload each podcast episode
#     points = []
#     for _, row in tqdm(podcast_df.iterrows(), total=len(podcast_df), desc="Embedding podcast episodes"):
#         episode_id = row['episode']
#         title = row['title'] if not pd.isna(row['title']) else ""
#         summary = row['summary'] if not pd.isna(row['summary']) else ""
        
#         # Create a text representation of the episode
#         text_representation = f"title:{title}; summary:{summary}"
        
#         # Embed the text
#         embedding = list(model.embed(text_representation))[0]
        
#         # Create a point
#         points.append(
#             models.PointStruct(
#                 id=int(episode_id),
#                 vector=embedding,
#                 payload={
#                     'episode': int(episode_id),
#                     'title': title,
#                     'summary': summary
#                 }
#             )
#         )
    
#     # Upload points in batches
#     batch_size = 100
#     for i in range(0, len(points), batch_size):
#         batch = points[i:i+batch_size]
#         client.upsert(
#             collection_name=collection_name,
#             points=batch
#         )
    
#     print(f"Uploaded {len(points)} episodes to Qdrant.")
#     return client

In [29]:
with open('podcast_ground_truth.json', 'r', encoding='utf-8') as f:
    ground_truth = json.load(f)

In [31]:
# Step 4: Evaluate retrieval using ground truth sentences
def evaluate_retrieval(client, model, ground_truth, collection_name="podcast_topic_collection", k=5):
    """
    Evaluate retrieval performance using ground truth sentences.
    
    For each sentence in the ground truth data, embed it and query Qdrant.
    Check if the correct episode is in the top k results.
    """
    # Prepare to store results
    all_relevance = []
    
    # Process each episode and its sentences
    for episode_id, episode_data in tqdm(ground_truth.items(), desc="Evaluating episodes"):
        sentences = episode_data.get('sentence', [])
        
        for sentence in sentences:
            # Embed the sentence
            query_vector = list(model.embed(sentence))[0]
            
            # Query Qdrant
            search_results = client.search(
                collection_name=collection_name,
                query_vector=query_vector,
                limit=k
            )
            
            # Check if the correct episode is in the results
            result_ids = [result.id for result in search_results]
            relevance = [int(episode_id) == result_id for result_id in result_ids]
            
            all_relevance.append(relevance)
    
    return all_relevance

In [26]:
# Step 5: Calculate metrics
def calculate_metrics(relevance_results):
    """Calculate hit rate and MRR from relevance results."""
    # Hit rate (Recall@k)
    hit_count = sum(1 for relevance in relevance_results if True in relevance)
    hit_rate = hit_count / len(relevance_results) if relevance_results else 0
    
    # Mean Reciprocal Rank (MRR)
    mrr_sum = 0
    for relevance in relevance_results:
        for i, is_relevant in enumerate(relevance):
            if is_relevant:
                mrr_sum += 1 / (i + 1)
                break
    
    mrr = mrr_sum / len(relevance_results) if relevance_results else 0
    
    return {
        'hit_rate': hit_rate,
        'mrr': mrr,
        'total_queries': len(relevance_results),
        'successful_queries': hit_count
    }


In [33]:
# Step 6: Evaluate for different k values
def evaluate_multiple_k(client, model, ground_truth, collection_name="podcast_topic_collection", k_values=[3, 5, 10]):
    """Evaluate retrieval for different k values."""
    results = {}
    
    for k in k_values:
        print(f"\nEvaluating for k={k}")
        relevance_results = evaluate_retrieval(client, model, ground_truth, collection_name, k)
        metrics = calculate_metrics(relevance_results)
        
        results[k] = metrics
        print(f"Hit Rate: {metrics['hit_rate']:.4f}, MRR: {metrics['mrr']:.4f}")
    
    return results

In [34]:
# Evaluate for different k values
print("Evaluating search performance...")
results = evaluate_multiple_k(client, model, ground_truth)

# Print final results
print("\nFinal Evaluation Results:")
print("========================")
for k, metrics in results.items():
    print(f"k={k}:")
    print(f"  Hit Rate: {metrics['hit_rate']:.4f}")
    print(f"  MRR:      {metrics['mrr']:.4f}")
    print(f"  ({metrics['successful_queries']} out of {metrics['total_queries']} queries successful)")


Evaluating search performance...

Evaluating for k=3


Evaluating episodes:   0%|                                                            | 0/411 [00:00<?, ?it/s]/var/folders/f5/c2cthfkx51zdsrmg13dv_kwh0000gn/T/ipykernel_32524/684283197.py:21: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(
Evaluating episodes: 100%|██████████████████████████████████████████████████| 411/411 [03:50<00:00,  1.78it/s]


Hit Rate: 0.2092, MRR: 0.1672

Evaluating for k=5


Evaluating episodes: 100%|██████████████████████████████████████████████████| 411/411 [04:06<00:00,  1.67it/s]


Hit Rate: 0.2511, MRR: 0.1768

Evaluating for k=10


Evaluating episodes: 100%|██████████████████████████████████████████████████| 411/411 [03:07<00:00,  2.20it/s]

Hit Rate: 0.3148, MRR: 0.1855

Final Evaluation Results:
k=3:
  Hit Rate: 0.2092
  MRR:      0.1672
  (430 out of 2055 queries successful)
k=5:
  Hit Rate: 0.2511
  MRR:      0.1768
  (516 out of 2055 queries successful)
k=10:
  Hit Rate: 0.3148
  MRR:      0.1855
  (647 out of 2055 queries successful)
